## Sprawozdanie1

**Data:** 2024-05-19  
**Autor:** Piotr Szepietowski

## Zadania działu 1

## Zadanie 1
Utwórz macierze **A** i **B** oraz wektor **f** określone jako:
$$
A = \begin{bmatrix}
2 & -7 \\
5 & 4 \\
\end{bmatrix} 
B = \begin{bmatrix}
6 & 1 \\
4 & -3 \\
\end{bmatrix} 
f = \begin{bmatrix}
4 \\
1 \\
\end{bmatrix} 
$$

In [14]:
import numpy as np

A = np.array([[2, -7], [5, 4]])
B = np.array([[6, 1], [4, -3]])
f = np.array([[4, 1]])

print(f"A = {A}")
print(f"B = {B}")
print(f"f = {f}")

A = [[ 2 -7]
 [ 5  4]]
B = [[ 6  1]
 [ 4 -3]]
f = [[4 1]]


## Zadanie 2

**a.** Wyświetl rozmiar macierzy **A** oraz wektora **f**.  
**b.** Dokonaj transpozycji macierzy **B**.  
**c.** Oblicz wyrażenie: $(A + B)^2 + 2(A - B)$  
**d.** Utwórz macierz $C = [A, B]$ oraz wektor $h = [f, f]$  
**e.** Oblicz iloczyn $C \cdot h$

In [15]:
# a
print(f"Rozmiar macierzy A: {A.shape}")
print(f"Rozmiar wektora f: {f.shape}")

# b
print(f"Macierz B po transpozycji:\n{B.T}")

# c
print(f"Wynik operacji (A + B)^2 + 2(A - B):\n{np.linalg.matrix_power(A + B, 2) + 2 * (A - B)}")

# d
C = np.vstack((A, B))
print(f"Macierz C = np.vstack((A, B))\n{C}")
h = np.vstack((f.T, f.T))
print(f"Wektor h = np.vstack((f.T, f.T))\n{h}")

# e
print(f"Macierz C.T * h:\n{C.T @ h}\nNiemożliwe do obliczenia, ponieważ liczba kolumn macierzy C nie zgadza się z liczbą wierszy wektora h, więc mnożenie wykonano po transpozycji macierzy C.")

Rozmiar macierzy A: (2, 2)
Rozmiar wektora f: (1, 2)
Macierz B po transpozycji:
[[ 6  4]
 [ 1 -3]]
Wynik operacji (A + B)^2 + 2(A - B):
[[  2 -70]
 [ 83 -39]]
Macierz C = np.vstack((A, B))
[[ 2 -7]
 [ 5  4]
 [ 6  1]
 [ 4 -3]]
Wektor h = np.vstack((f.T, f.T))
[[4]
 [1]
 [4]
 [1]]
Macierz C.T * h:
[[ 41]
 [-23]]
Niemożliwe do obliczenia, ponieważ liczba kolumn macierzy C nie zgadza się z liczbą wierszy wektora h, więc mnożenie wykonano po transpozycji macierzy C.


## Zadanie 3

**a.** Wyświetl wszystkie zmienne lokalne.  
**b.** Zapisz zmienne w pliku `dane`.  
**c.** Usuń wszystkie zmienne.  
**d.** Wyświetl zawartość katalogu roboczego.

In [16]:
import os

# a. Wyświetl wszystkie zmienne lokalne
print("Zmienne lokalne:", list(locals().keys()))

# b. Zapisz zmienne w pliku 'dane' jako tekst
with open('dane', 'w') as f_out:
    f_out.write(f"Zmienne lokalne:\n")
    for x in list(locals().keys()):
        f_out.write(f"{x}\n")

# c. Usuń wszystkie zmienne
vars_to_delete = [var for var in globals().keys() if not var.startswith("_") and var not in ["pickle", "f_out", "os", "types"]]
for var in vars_to_delete:
    del globals()[var]

# d. Wyświetl zawartość katalogu roboczego
print(f"Zawartość katalogu domowego to: {os.listdir()}")

Zmienne lokalne: ['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', '_i2', '_i3', '_i4', '_i5', 'os', 'f_out', 'vars_to_delete', 'var', '_i6', 'np', 'x', 'Y', '_i7', 'math', 'e', '_i8', 'z', '_i9', 'ones', 'zeroes', 'random', '_i10', 'A', 'y', '_i11', 'i', 'j', '_i12', '_i13', 'display', 'n', '_i14', 'B', 'f', '_i15', 'C', 'h', '_i16']
Zawartość katalogu domowego to: ['.git', '.venv', 'dane', 'Sprawozdanie1.ipynb', 'Sprawozdanie2.ipynb', 'Sprawozdanie3.ipynb']


## Zadanie 4

**a.** Utwórz 24-elementowy wektor $x = [1, 2, \ldots, 24]$.  
**b.** Z wektora $x$ utwórz macierz $Y$ o postaci $6 \times 4$.

In [17]:
import numpy as np
x = np.array([x for x in range(1, 25)])
print(f"x = {x}")

Y = x.reshape(4, 6)
print(f"Y = {Y}")


x = [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
Y = [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]
 [19 20 21 22 23 24]]


## Zadanie 5

Oblicz:  
**a.** $e^{2\sin(2\pi)}$  
**b.** $\cos\left(\frac{\pi}{3}\right)^4$  
**c.** $\ln(\sqrt{5})$

In [18]:
import math
e = math.e
print(f"a = {e ** (2 * math.sin(2 * math.pi))}")
print(f"b = {math.cos(math.pi / 3) ** 4}")
print(f"c = {math.log(math.sqrt(5))}")

a = 0.9999999999999996
b = 0.06250000000000006
c = 0.8047189562170503


## Zadanie 6
Oblicz:  
**a.** moduł liczby zespolonej $z = 3 - 2j$  
**b.** argument $z$  
**c.** liczba zespolona sprzężona do $z$

In [19]:
z = 3 - 2j
print(f"z = {z}")
print(f"Mod(z) = {abs(z)}")
print(f"Arg(z) = {math.atan2(z.imag, z.real)}")
print(f"Sprzężona z = {z.conjugate()}")

z = (3-2j)
Mod(z) = 3.605551275463989
Arg(z) = -0.5880026035475675
Sprzężona z = (3+2j)


## Zadanie 7
Utwórz macierze o rozmiarze 3x4: $$ $$
**a.**  
Macierz o elementach równych 1:  
$$
\begin{bmatrix}
1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 \\
\end{bmatrix}
$$

**b.**  
Macierz o elementach równych 0:  
$$
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{bmatrix}
$$

**c.**  
Macierz o argumentach pseudolosowych:  
$$
\begin{bmatrix}
x_{11} & x_{12} & x_{13} & x_{14} \\
x_{21} & x_{22} & x_{23} & x_{24} \\
x_{31} & x_{32} & x_{33} & x_{34} \\
\end{bmatrix}
$$  
gdzie $x_{ij}$ to liczby pseudolosowe.

In [20]:
ones = np.ones((3, 4))
print(f"Macierz jedynek:\n{ones}")
zeroes = np.zeros((3, 4))
print(f"Macierz zer:\n{zeroes}")
random = np.random.rand(3, 4)
print(f"Macierz pseudolosowych liczb:\n{random}")

Macierz jedynek:
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
Macierz zer:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Macierz pseudolosowych liczb:
[[0.95209829 0.82598429 0.3315324  0.31462675]
 [0.28930604 0.93560357 0.33290845 0.7804975 ]
 [0.70909999 0.94944113 0.18082626 0.26733682]]



---
## Zadania działu 2
## Zadanie 1
Utwórz macierz **A** o rozmiarze 6x6, gdzie  
$$
x_{ij} = 
\begin{cases}
\frac{1}{i-j} & \text{dla } i \neq j \\
0 & \text{dla } i = j
\end{cases}
$$

In [21]:
A = np.zeros((6, 6))
for x in range(6):
    for y in range(6):
        if x != y:
            A[x, y] = 1 / ( x - y )
print(f"Macierz A:\n{A}")

Macierz A:
[[ 0.         -1.         -0.5        -0.33333333 -0.25       -0.2       ]
 [ 1.          0.         -1.         -0.5        -0.33333333 -0.25      ]
 [ 0.5         1.          0.         -1.         -0.5        -0.33333333]
 [ 0.33333333  0.5         1.          0.         -1.         -0.5       ]
 [ 0.25        0.33333333  0.5         1.          0.         -1.        ]
 [ 0.2         0.25        0.33333333  0.5         1.          0.        ]]


## Zadanie 2
Wygeneruj macierz o rozmiarze 10x10 wypełnioną liczbami pseudolosowymi, wyświetl elementy macierzy z przedziału (0.2-0.5).

In [22]:
A = np.random.rand(10, 10)
for i in range(10):
    for j in range(10):
        if A[i, j] < 0.5 and A[i, j] > 0.2:
            print(f"A[{i}, {j}] = {A[i, j]}")

A[0, 2] = 0.4864259717785895
A[1, 4] = 0.27045481522920056
A[1, 7] = 0.4115861243786135
A[2, 1] = 0.46203290197118374
A[2, 3] = 0.31263702296628415
A[2, 4] = 0.27256638200211425
A[3, 0] = 0.3010431561848387
A[3, 2] = 0.35754359983358497
A[3, 5] = 0.22588457197279954
A[3, 8] = 0.26836139072749643
A[4, 0] = 0.4568444461870932
A[4, 3] = 0.4119729012884966
A[4, 6] = 0.4899857253278559
A[4, 8] = 0.24267532204699616
A[4, 9] = 0.23038859571486425
A[5, 0] = 0.3864799123519358
A[5, 1] = 0.2385951725683776
A[5, 5] = 0.2102934303416356
A[6, 2] = 0.45668582943876046
A[6, 4] = 0.45582552324353887
A[7, 4] = 0.4492061464641832
A[7, 9] = 0.23121297464719348
A[8, 6] = 0.4963989390864719
A[9, 1] = 0.41349354387558257
A[9, 7] = 0.22761333931523864
A[9, 9] = 0.35906551385686025


## Zadanie 3
Napisz skrypt w języku Python, który:

1. W pierwszych wierszach zawiera komentarz:
   
   ```
   # Skrypt ten jest skryptem testowym
   # Obliczane jest wyrazenie sin(a)*cos(b)
   # gdzie a, b - dwie wczytane liczby
   ```

2. Wczytuje dwie liczby `a` i `b` od użytkownika za pomocą funkcji `input()`.

3. Oblicza wartość wyrażenia `sin(a)*cos(b)` i wyświetla wynik.

In [23]:
# Skrypt ten jest skryptem testowym
# Obliczane jest wyrazenie sin(a)*cos(b)
# gdzie a, b - dwie wczytane liczby

# Zostaje zakomentowane ponieważ w jupyterze nie można używać input

#a = float(input("Podaj liczbę a: "))
#b = float(input("Podaj liczbę b: "))

#wynik = math.sin(a) * math.cos(b)
#print(f"Wynik: {wynik}")

## Zadanie 4

Napisz skrypt, którego zadaniem będzie prezentacja operacji na macierzach. W skrypcie wykonaj następujące polecenia:

**a.** wczytaj:
   - rozmiar macierzy (n),
   - dwie macierze o rozmiarze n×n: A i B,

**b.** oblicz:
   - macierz odwrotną do A,
   - transpozycję macierzy B,
   - sumę macierzy – A+B,
   - różnicę macierzy – A-B,
   - iloczyn macierzowy – A·B,
   - iloczyn macierzy A przez dowolną liczbę.

In [24]:
def display(n):
    A = np.random.rand(n, n)
    B = np.random.rand(n, n)
    print(f"Macierz A:\n{A}\nMacierz B:\n{B}")
    print(f"A odwrócona:\n{np.linalg.inv(A)}")
    print(f"B po transpozycji:\n{B.T}")
    print(f"Suma A + B:\n{A + B}")
    print(f"Różnica A - B:\n{A - B}")
    print(f"Iloczyn A * B:\n{A @ B}")
    print(f"Iloczyn macierzy A przez skalara 5:\n{A * 5}")    

n = 3
display(n)

Macierz A:
[[0.6345005  0.44175051 0.96595947]
 [0.69553147 0.55870423 0.6236622 ]
 [0.47891603 0.75611226 0.83867127]]
Macierz B:
[[0.63733517 0.31136622 0.91454876]
 [0.84687862 0.25042586 0.85211705]
 [0.87680645 0.59738913 0.3274754 ]]
A odwrócona:
[[-0.02452438  2.95241363 -2.16726064]
 [-2.33508993  0.57034933  2.26536543]
 [ 2.11922751 -2.20015446  0.3875956 ]]
B po transpozycji:
[[0.63733517 0.84687862 0.87680645]
 [0.31136622 0.25042586 0.59738913]
 [0.91454876 0.85211705 0.3274754 ]]
Suma A + B:
[[1.27183567 0.75311672 1.88050823]
 [1.54241009 0.8091301  1.47577925]
 [1.35572248 1.35350139 1.16614668]]
Różnica A - B:
[[-0.00283467  0.13038429  0.05141071]
 [-0.15134715  0.30827837 -0.22845485]
 [-0.39789042  0.15872313  0.51119587]]
Iloczyn A * B:
[[1.62545804 0.88524146 1.27303275]
 [1.46327238 0.72904802 1.31641288]
 [1.68091772 0.83948144 1.35693242]]
Iloczyn macierzy A przez skalara 5:
[[3.17250251 2.20875253 4.82979735]
 [3.47765735 2.79352116 3.11831102]
 [2.39458016 3.

## Zadanie 5
Oblicz i porównaj czas wykonania dla dwóch metod:

**a. Obliczenia w pętli (element po elemencie):**  
Dla każdego punktu z przedziału od -10 do 10 z krokiem 0,001 oblicz wartość funkcji cosinus, zapisując wyniki do listy. Zmierz czas wykonania tej operacji.

**b. Obliczenia tablicowe (jednorazowo):**  
Oblicz wartości funkcji cosinus dla wszystkich punktów z przedziału od -10 do 10 z krokiem 0,001 jednocześnie, wykorzystując operacje na tablicach. Zmierz czas wykonania tej operacji.

In [26]:
import time

x_points = np.arange(-10, 10.001, 0.001)

start_loop = time.time()
y_loop = []
for xi in x_points:
    y_loop.append(np.cos(xi))
end_loop = time.time()
print(f"Czas obliczeń w pętli: {end_loop - start_loop:.4f} s")

start_vec = time.time()
y_vec = np.cos(x_points)
end_vec = time.time()
print(f"Czas obliczeń tablicowych: {end_vec - start_vec:.4f} s")

Czas obliczeń w pętli: 0.0343 s
Czas obliczeń tablicowych: 0.0004 s


## Zadanie 2.6

Napisz funkcję **metrs**, dokonującą przeliczenia prędkości wyrażonej w [km/h] na prędkość w [m/s].

In [27]:
def metrs(velocity):
    return velocity / 3.6

print(f"Prędkość 90 km/h w m/s: {metrs(90)}")

Prędkość 90 km/h w m/s: 25.0



---
## Zadania działu 3
## Zadanie 1
Zapisz w pliku dane.bin 50-elementowy wektor liczb pseudolosowych.